In [1]:
api_key="de9bf1c4-a94f-4f9a-9c00-0935b36f7237"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
from pinecone import Pinecone,ServerlessSpec

In [5]:
index_name='hybrid-search-langchain-pinecone'
pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws',region='us-east-1'),
    )

In [6]:
index=pc.Index(index_name)

e:\genai_by_krishnaik\envi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
index

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [17]:
sentences=[
    'In 2023, i visited paris',
        'in 2022, i visited new york',
        'in 2021, i visited new orleans'
]

bm25_encoder.fit(sentences)

bm25_encoder.dump('bm25_values.json')

bm25_encoder=BM25Encoder().load('bm25_values.json')

100%|██████████| 3/3 [00:00<00:00, 2991.66it/s]


In [18]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [19]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001CF68205060>, index=<pinecone.db_data.index.Index object at 0x000001CF67B65ED0>)

In [20]:
retriever.add_texts(
    [
       'In 2023, i visited paris',
        'in 2022, i visited new york',
        'in 2021, i visited new orleans', 
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.97s/it]


In [21]:
retriever.invoke('what city did i visit last')

[Document(metadata={'score': 0.270718694}, page_content='in 2021, i visited new orleans'),
 Document(metadata={'score': 0.255977273}, page_content='in 2022, i visited new york'),
 Document(metadata={'score': 0.235791892}, page_content='In 2023, i visited paris')]